In [1]:
# import Librarairs
import os,json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve 

In [2]:
# Access Data
CWD = os.getcwd()  # get current directory path
BUG_REPORTS_DIR = os.path.join(CWD,"bugs")   # current directory+bugs
BUG_REPORTS_LIST = os.listdir(BUG_REPORTS_DIR)  # all the items in 'current_directory+bugs'

In [3]:

SENTI_REPORTS_DIR = os.path.join(CWD,"Sentiments_output")
SENTI_REPORTS_LIST = os.listdir(SENTI_REPORTS_DIR)

In [4]:
# Load data set for bugs and sentiments


print(BUG_REPORTS_DIR)
print('-----------------')
print(BUG_REPORTS_LIST)
print('-----------------')
print(SENTI_REPORTS_DIR)
print('-----------------')
print(SENTI_REPORTS_LIST)         

/home/cr/Desktop/ali/bugs
-----------------
['Core.json', 'Toolkit.json', 'bugzilla.json', 'Calendar.json', 'SeaMonkey.json', 'Camino Graveyard.json', 'MailNews Core.json', 'Firefox.json', 'Thunderbird.json', 'Core Graveyard.json']
-----------------
/home/cr/Desktop/ali/Sentiments_output
-----------------
['Camino_Graveyard_sentiments.csv', 'bugzilla_sentiments.csv', 'Firefox_sentiments.csv', 'Core_sentiments.csv', 'MailNews_Core_sentiments.csv', 'SeaMonkey_sentiments.csv', 'Thunderbird_sentiments.csv', 'Toolkit_sentiments.csv', 'Core_Graveyard_sentiments.csv', 'Calendar_sentiments.csv']


In [29]:
# Extract data from files for sentiments
import csv
test_senti = []
train_senti=[]
import pandas as pd
for count, report in enumerate(SENTI_REPORTS_LIST):
    
    if count == 3:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data['neutral'])
            test_senti.extend(temp_data.iloc[0:, 1])
            
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data.iloc[0:, 1])
            train_senti.extend(temp_data.iloc[0:, 1])
            
            #print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")


Train Set: Camino_Graveyard_sentiments.csv
1291
Train Set: bugzilla_sentiments.csv
5325
Train Set: Firefox_sentiments.csv
7369
Test Set: Core_sentiments.csv
7229
Train Set: MailNews_Core_sentiments.csv
2797
Train Set: SeaMonkey_sentiments.csv
8717
Train Set: Thunderbird_sentiments.csv
8717
Train Set: Toolkit_sentiments.csv
1900
Train Set: Core_Graveyard_sentiments.csv
889
Train Set: Calendar_sentiments.csv
1786


In [107]:
# Load data for bugs 
train_set = []
test_set = []

for count, report in enumerate(BUG_REPORTS_LIST):
    if count == 9:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            test_set.extend(temp_data["bugs"])
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            train_set.extend(temp_data["bugs"])   

Train Set: Core.json
File:Core.json <> Reports: 7228
Train Set: Toolkit.json
File:Toolkit.json <> Reports: 1899
Train Set: bugzilla.json
File:bugzilla.json <> Reports: 5324
Train Set: Calendar.json
File:Calendar.json <> Reports: 1785
Train Set: SeaMonkey.json
File:SeaMonkey.json <> Reports: 8716
Train Set: Camino Graveyard.json
File:Camino Graveyard.json <> Reports: 1290
Train Set: MailNews Core.json
File:MailNews Core.json <> Reports: 2796
Train Set: Firefox.json
File:Firefox.json <> Reports: 7368
Train Set: Thunderbird.json
File:Thunderbird.json <> Reports: 5129
Test Set: Core Graveyard.json
File:Core Graveyard.json <> Reports: 888


In [108]:
print(f'Train len: {len(train_set)}')
print(f'Test len: {len(test_set)}')
print(f'Total len: {len(train_set)+len(test_set)}')

Train len: 41535
Test len: 888
Total len: 42423


In [109]:
summary_train=[]
label_train=[]
summary_test=[]
label_test=[]
for i in train_set:
    summary_train.append(i['summary'])
    #summary_train=summary_train[0:10]
    label_train.append(i['resolution'])
print('train_summary',len(summary_train))
print('train_label',len(label_train))

    #label_train=label_train[0:10]
for j in test_set:
    summary_test.append(j['summary'])
    #summary_test=summary_test[0:10]
    label_test.append(j['resolution'])
   # label_test=label_test[0:10]
print('test_summary',len(summary_test))
print('test_label',len(label_test))
description_train=[]
description_test=[]

for k in train_set:
    description_train.append(k['description'])

for l in test_set:
    description_test.append(l['description'])

train_summary 41535
train_label 41535
test_summary 888
test_label 888


In [110]:
#Preprocess sentences
import re
def train_test_summary(summaries_text):
    
    lemmas=[]
    lemmatized_word=[]
    train_test_summaries=[]
    for count, summaries in enumerate (summaries_text):
        text = re.sub(r'[^a-zA-Z\']', ' ', summaries)
        #removes all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        #substitute multiple spcaces with single space
        s_m_s = re.sub(r'\s+', ' ', document, flags=re.I)
        #removes prefix
        r_prefix = re.sub(r'^b\s+', '', s_m_s)
        #lemmas.append(r_prefix)
        r_pm = re.sub(r'[^\w\s]', '', r_prefix)
        step1 = r_pm.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
        step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
        step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
        step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
        step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
        step6 = step5.replace(" ` ", " '")
        step7=step6.strip()
        step8=step7.lower()    
        lemmas=[]
        sentences = nltk.word_tokenize(step7)
        lemmatizer = WordNetLemmatizer()
        stem_words=[]
        for sentence_words in sentences:
            lower=sentence_words.lower()
            if lower not in stopwords.words('english'):
                lemma = lemmatizer.lemmatize(lower)
                stem_words.append(lemma)
            untokenized_sentence= ' '.join(stem_words)
        train_test_summaries.append(untokenized_sentence)
    return train_test_summaries

In [111]:
# Encocede Labels 
def train_test_labels(labels):
    train_test_label=[]
    for x in range(len(labels)):
        
        
        if label_train[x]=="FIXED":
            label_train[x]=1
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="INVALID":
            label_train[x]=0
        elif label_train[x]=="INCOMPLETE":
            label_train[x]=0
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="WORKSFORME":
            label_train[x]=0
        elif label_train[x]=="EXPIRED":
            label_train[x]=0
        elif label_train[x]=="DUPLICATE":
            label_train[x]=0
        elif label_train[x]=="":
            label_train[x]=0
        train_test_label.append(label_train[x])
        
    
    
    #print('train_lbel', len(train_label))
    return train_test_label

In [112]:
train_labels= train_test_labels(label_train)
test_labels=train_test_labels(label_test)

train_labels=np.array(train_labels)
test_labels=np.array(test_labels)


print(f"Training Labels Length:{len(train_labels)}")
print(f"Testing Labels Length:{len(test_labels)}")

print(f'Total len: {len(train_labels)+len(test_labels)}')

# Use 
#     train_labels
#     test_labels
test_lab, counts_elements =np.unique(test_labels, return_counts=True)
print(np.asarray((test_lab, counts_elements)))

Training Labels Length:41535
Testing Labels Length:888
Total len: 42423
[[  0   1]
 [606 282]]


In [12]:

#train_summary=train_test_summary(summary_train)
#test_summary=train_test_summary(summary_test)

#print(f"Training summary Length:{len(train_summary)}")
#print(f"Testing summary Length:{len(test_summary)}")



# Use 
#     train_summary
#     test_summary

In [13]:

#d_train=train_test_summary(description_train)
#d_test=train_test_summary(description_test)
#print(f"Training summary Length:{len(d_train)}")
#print(f"Testing summary Length:{len(d_test)}")
#print(f'Total len: {len(d_train)+len(d_test)}')


In [113]:
vocab_size = 1000
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_l = 60 # max length of the sentence
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(summary_train)

word_index = tokenizer.word_index
print(word_index)

training_sequences = tokenizer.texts_to_sequences(summary_train)
training_padded1 = pad_sequences(training_sequences, maxlen=max_l, padding=padding_type, truncating=trunc_type)

print(training_padded1.shape)
testing_sequences = tokenizer.texts_to_sequences(summary_test)
testing_padded1= pad_sequences(testing_sequences, maxlen=max_l, padding=padding_type, truncating=trunc_type)
print(testing_padded1[1])
print(testing_padded1.shape)
print(type(testing_padded1))

{'<OOV>': 1, 'to': 2, 'in': 3, 'the': 4, 'for': 5, 'should': 6, 'a': 7, 'of': 8, 'and': 9, 'add': 10, 'on': 11, 'be': 12, 'rfe': 13, 'not': 14, 'new': 15, 'when': 16, 'from': 17, 'mail': 18, 'option': 19, 'with': 20, 'tab': 21, 'is': 22, 'as': 23, 'window': 24, 'search': 25, 'menu': 26, 'support': 27, 'message': 28, 'allow': 29, 'page': 30, 'open': 31, 'bug': 32, 'use': 33, 'or': 34, 'all': 35, 'bar': 36, 'an': 37, 'text': 38, 'by': 39, 'folder': 40, 'button': 41, 'make': 42, 'tabs': 43, 'list': 44, 'show': 45, 'file': 46, 'feature': 47, 'have': 48, 'user': 49, 'ability': 50, 'request': 51, 'address': 52, 'need': 53, 'default': 54, 'bookmarks': 55, 'no': 56, 'firefox': 57, 'messages': 58, 'mozilla': 59, 'implement': 60, 'view': 61, 'if': 62, 'url': 63, 'download': 64, 'email': 65, 'it': 66, 'manager': 67, 'bugzilla': 68, 'bookmark': 69, 'that': 70, 'link': 71, 'toolbar': 72, 'like': 73, 'dialog': 74, 'display': 75, 'multiple': 76, 'click': 77, 'context': 78, 'windows': 79, 'only': 80, 

(41535, 60)
[  1   1 201 369   5   1   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0]
(888, 60)
<class 'numpy.ndarray'>


In [114]:
vocab_size = 1000
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len=60 # max length of the sentence
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(description_train)

word_index = tokenizer.word_index
#print(word_index)

training_sequences = tokenizer.texts_to_sequences(description_train)
training_padded2 = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

print(training_padded2.shape)
testing_sequences = tokenizer.texts_to_sequences(description_test)
testing_padded2 = pad_sequences(testing_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print(testing_padded2[1])
print(testing_padded2.shape)
print(type(testing_padded2))

(41535, 60)
[197 617  42 341   1 973  11  10   1 193  68  85   1 496   1  13   1   1
   9   1 154   4 104   3 473   2 722 187   4 686 294   9 111  35   1   3
   2 180   1  11   1  18  37  13 553   8  44   1 104   0   0   0   0   0
   0   0   0   0   0   0]
(888, 60)
<class 'numpy.ndarray'>


In [22]:
senti_summary_train= np.append(training_padded1,training_padded2,axis=1)
senti_summary_test= np.append(testing_padded1,testing_padded2,axis=1)
X1=senti_summary_train
Y1=senti_summary_test

print(X1.shape)
print(Y1.shape)

(37099, 120)
(5324, 120)


In [115]:
# CNN
from keras.models import Sequential
from keras import layers
from keras.layers import Flatten
from keras.optimizers import SGD
embedding_dim = 256

with tf.device('/GPU:0'):


    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=60))
    model.add(layers.Conv1D(32, 1, activation='tanh'))
    
    #model.add(layers.Conv1D(2, 1, activation='tanh')) # best 
    #model.add(layers.GlobalMaxPooling1D())
    #model.add(Flatten())
    model.add(layers.AveragePooling1D())
    model.add(layers.BatchNormalization())
    
    #model.add(layers.Conv1D(2, 1, activation='relu'))
    #model.add(layers.BatchNormalization())
    #model.add(layers.GRU(16))
    #model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    


    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 60, 256)           256000    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 60, 32)            8224      
_________________________________________________________________
average_pooling1d_8 (Average (None, 30, 32)            0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 32)            128       
_________________________________________________________________
dense_25 (Dense)             (None, 30, 10)            330       
_________________________________________________________________
dense_26 (Dense)             (None, 30, 1)             11        
Total params: 264,693
Trainable params: 264,629
Non-trainable params: 64
______________________________________________

In [26]:
# CNN
from keras.models import Sequential
from keras import layers
from keras.layers import Flatten
from keras.optimizers import SGD
embedding_dim = 256

from keras.layers import *



model1 = Sequential(layers=[
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    #GRU(300,return_sequences=True),
    #BatchNormalization(),
    Conv1D(32, 1, activation='tanh'),
    AveragePooling1D(),
    BatchNormalization(),

])

model2 = Sequential(layers=[
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    #GRU(300,return_sequences=True),
    #BatchNormalization(),
    Conv1D(32, 1, activation='tanh'),
    AveragePooling1D(),
    BatchNormalization(),

])

mergedOut = Add()([model1.output,model2.output])



mergedOut = BatchNormalization()(mergedOut)    
mergedOut = Dense(128, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(64, activation='relu')(mergedOut)
mergedOut = Dropout(.35)(mergedOut)

mergedOut = Dense(1, activation='sigmoid')(mergedOut)

# output layer
#mergedOut = Dense(5, activation='softmax')(mergedOut)


from keras.models import Model

newModel = Model([model1.input,model2.input], mergedOut)



newModel.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
newModel.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_5_input (InputLayer)  [(None, 60)]         0                                            
__________________________________________________________________________________________________
embedding_6_input (InputLayer)  [(None, 60)]         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 60, 256)      256000      embedding_5_input[0][0]          
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 60, 256)      256000      embedding_6_input[0][0]          
____________________________________________________________________________________________

In [27]:
with tf.device('/GPU:0'):
    num_epochs = 10
    batch_size = 32
    newModel.fit([training_padded1, training_padded2],train_labels, validation_split=0.30, epochs=num_epochs, batch_size=batch_size, verbose=1)

Epoch 1/10
812/812 [==============================] - 258s 317ms/step - loss: 0.4397 - accuracy: 0.8128 - val_loss: 0.2807 - val_accuracy: 0.9006
Epoch 2/10
812/812 [==============================] - 258s 318ms/step - loss: 0.4008 - accuracy: 0.8276 - val_loss: 0.2668 - val_accuracy: 0.9018
Epoch 3/10
812/812 [==============================] - 266s 327ms/step - loss: 0.3797 - accuracy: 0.8369 - val_loss: 0.2896 - val_accuracy: 0.9036
Epoch 4/10
812/812 [==============================] - 273s 336ms/step - loss: 0.3606 - accuracy: 0.8447 - val_loss: 0.2819 - val_accuracy: 0.9027
Epoch 5/10
812/812 [==============================] - 276s 340ms/step - loss: 0.3411 - accuracy: 0.8532 - val_loss: 0.3063 - val_accuracy: 0.9012
Epoch 6/10
812/812 [==============================] - 305s 376ms/step - loss: 0.3198 - accuracy: 0.8608 - val_loss: 0.3355 - val_accuracy: 0.9009
Epoch 7/10
812/812 [==============================] - 331s 408ms/step - loss: 0.2974 - accuracy: 0.8683 - val_loss: 0.3465 -

In [28]:
# evaluate the model
_, train_accuracy = newModel.evaluate([training_padded1,training_padded2], train_labels, verbose=0)
_, test_accuracy = newModel.evaluate([testing_padded1,testing_padded2],test_labels, verbose=0)
print(f"Train accuracy RNN:{train_accuracy*100}")
print(f"Test Accracy RNN:{test_accuracy*100}")



Train accuracy RNN:91.51694178581238
Test Accracy RNN:66.35172963142395


In [116]:
with tf.device('/GPU:0'):
    num_epochs = 1
    batch_size = 32
    history = model.fit(training_padded1, train_labels, epochs=num_epochs, batch_size=batch_size, verbose=1)

1298/1298 [==============================] - 16s 12ms/step - loss: 0.4930 - accuracy: 0.8052


In [119]:
# evaluate the model
_, train_accuracy = model.evaluate(training_padded1, train_labels, verbose=0)
_, test_accuracy = model.evaluate(testing_padded1,test_labels, verbose=0)
print(f"Train accuracy RNN:{train_accuracy*100}")
print(f"Test Accracy RNN:{test_accuracy*100}")

'''
Train accuracy RNN:80.70545792579651
Test Accracy RNN:68.21509003639221
'''

Train accuracy RNN:80.71338534355164
Test Accracy RNN:68.06305646896362


'\nTrain accuracy RNN:80.70545792579651\nTest Accracy RNN:68.21509003639221\n'

In [120]:
import numpy as np
yhat_prob = model.predict(testing_padded1)
#print(yhat_prob)

# get classes from probs
#yhat_class = np.argmax(yhat_prob, axis=-1) #for multilabel with softmax
yhat_class = (yhat_prob > 0.5).astype("int32") #for binary classes with sigmoid
#print(yhat_class[0:10])
# flaten pridicted and test lables into 1d-array
yhat = yhat_class[:,0]

print('done prepare yhat_prob & yhat_class')
#print(yhat_class[0:10])
#print(test_labels[0:10])


done prepare yhat_prob & yhat_class


In [121]:
from sklearn.metrics import confusion_matrix
print(classification_report(test_labels,yhat))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels,yhat))
matrix = metrics.confusion_matrix(test_labels, yhat)
print(matrix)

              precision    recall  f1-score   support

           0       0.68      0.96      0.80       606
           1       0.33      0.05      0.08       282

    accuracy                           0.67       888
   macro avg       0.50      0.50      0.44       888
weighted avg       0.57      0.67      0.57       888

matthews_corrcoef: 0.0034674893737410657
[[579  27]
 [269  13]]


In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
accuracy = metrics.accuracy_score(test_labels, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = metrics.precision_score(test_labels, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = metrics.recall_score(test_labels, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = metrics.f1_score(test_labels, yhat)
print('F1 score: %f' % f1)
 
# kappa
kappa = metrics.cohen_kappa_score(test_labels, yhat)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = metrics.roc_auc_score(test_labels, yhat)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = metrics.confusion_matrix(test_labels, yhat)
print(matrix)

print(classification_report(y_true=test_labels, y_pred=yhat, digits=5))

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(training_padded, label_train)
pred=clf.predict(testing_padded)

In [ ]:
#print(pred)
#print(test_label)
accuracy = metrics.accuracy_score(label_test, pred)
print('Accuracy: %f' % accuracy)
precision = metrics.precision_score(label_test, pred)
print('Precision: %f' % precision)
recall = metrics.recall_score(label_test, pred)
print('Recall: %f' % recall)
print(classification_report(y_true=label_test, y_pred=pred, digits=5))

In [ ]:
import numpy as np
rng = np.random.RandomState(1)
from sklearn.naive_bayes import MultinomialNB
clfm = MultinomialNB()
clfm.fit(training_padded, label_train)
pred=clfm.predict(testing_padded)
accuracy = metrics.accuracy_score(label_test, pred)
print('Accuracy: %f' % accuracy)
precision = metrics.precision_score(label_test, pred)
print('Precision: %f' % precision)
recall = metrics.recall_score(label_test, pred)
print('Recall: %f' % recall)
print(classification_report(y_true=label_test, y_pred=pred, digits=5))